In [ ]:
# https://youtu.be/QogFIWa1YMg?list=PLA0M1Bcd0w8ynD1umfubKq1OBYRXhXkmH
%env TF_CPP_MIN_LOG_LEVEL=2

import tensorflow as tf  # noqa: E402
from tensorflow.keras.datasets import mnist  # noqa: E402
from tensorflow.keras.utils import to_categorical  # noqa: E402

# Load MNIST dataset

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess data
x_train = tf.cast(x_train.reshape((-1, 28 * 28)) / 255.0, tf.float32)
x_test = tf.cast(x_test.reshape((-1, 28 * 28)) / 255.0, tf.float32)

y_train = to_categorical(y_train)



In [ ]:
class DenseNN(tf.Module):
    """Dense neural network
    """
    def __init__(self, outputs_size: int, activate="relu"):
        """ctor

        Args:
            outputs_size (int): count inputs in begin layer
            activate (str, optional): activation function "relu" for tf.nn.relu or "softmax" for tf.nn.softmax . Defaults to "relu".
        """
        super().__init__()
        self.outputs_size = outputs_size
        self.activate = activate
        self.lf_init = False

    def __call__(self, x):
        if not self.lf_init:
            self.w = tf.random.truncated_normal([x.shape[-1], self.outputs_size], stddev=0.1, name='w')
            self.b = tf.zeros([self.outputs_size], dtype=tf.float32, name='b')

            self.w = tf.Variable(self.w)
            self.b = tf.Variable(self.b)

            self.lf_init = True

        y = x @ self.w + self.b
        if self.activate == "relu":
            return tf.nn.relu(y)
        elif self.activate == "softmax":
            return tf.nn.softmax(y)
        return y


In [ ]:

class SequentialModule(tf.Module):
    def __init__(self, name="MNIST_model"):
        super().__init__(name=name)
        self.dense1 = DenseNN(128)
        self.dense2 = DenseNN(10, activate="softmax")

    def __call__(self, x):
        return self.dense2(self.dense1(x))

model = SequentialModule()

# model.submodules

In [ ]:
def cross_entropy(y_pred, y_true):
    loss = tf.losses.categorical_crossentropy(y_true, y_pred)
    return tf.reduce_mean(loss)

# opt = tf.optimizers.Adam(learning_rate=0.001)
# for M1/M2 Macs architecture use legacy optimizer
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)

BATCH_SIZE = 32
EPOCHS = 20
TOTAL = x_train.shape[0]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

@tf.function
def train_batch(x_batch, y_batch):
    with tf.GradientTape() as tape:
            f_loss = cross_entropy(model(x_batch), y_batch)

    grad = tape.gradient(f_loss, model.trainable_variables)
    opt.apply_gradients(zip(grad, model.trainable_variables))
    return f_loss

for epoch in range(EPOCHS):
    loss = 0.0
    for x_batch, y_batch in train_dataset:
        loss += train_batch(x_batch, y_batch)

    print(f"Epoch {epoch+1}/{EPOCHS} loss {loss:.4f}")


In [ ]:
# Test model

y_pred = model(x_test)
y2 = tf.argmax(y_pred, axis=1).numpy()
acc = len(y_test[y2 == y_test]) / y_test.shape[0] * 100
print(f"Accuracy: {acc:.2f}%")

# # Get accuracy by Tensorflow
# acc = tf.metrics.Accuracy()
# acc.update_state(y_test, y2)
# print(f"Accuracy: {acc.result().numpy() * 100:.2f}%")

# Save model
# tf.saved_model.save(layer_1, "layer_1")
# tf.saved_model.save(layer_2, "layer_2")
tf.saved_model.save(model, "model")
